<a href="https://colab.research.google.com/github/jiyeonjin/0624_new/blob/main/project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===================================================================
# [ 최종 해결 코드 v17 ] - Processor 통과 후 최종 텐서 클렌징 (오류 수정됨)
# ===================================================================

# ----------------- [ 0. 환경 설정, 라이브러리 설치 (수정됨) ] -----------------
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # 디버깅 환경 설정

# [수정 사항] 라이브러리 버전 제한을 제거하여 pip가 현재 환경에 맞는
# 컴파일이 필요 없는 pre-built wheel 버전을 자동으로 찾도록 합니다.
# 이것이 "can't find Rust compiler" 오류를 해결하는 가장 일반적인 방법입니다.
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q "transformers" "datasets" roboflow accelerate opencv-python ipywidgets tqdm pillow "ipywidgets>=7.0.0"

# --- 라이브러리 임포트는 설치가 모두 끝난 후 진행합니다 ---
import glob, torch, cv2
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
from datasets import Dataset, DatasetDict, Image as HFImage
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation, TrainingArguments, Trainer
import ipywidgets as widgets
from IPython.display import display, clear_output
from roboflow import Roboflow
from torchvision.transforms import ColorJitter

print("✅ 라이브러리 로딩 및 디버깅 환경 설정 완료")


# ----------------- [ 1. 데이터셋 다운로드 ] -----------------
ROBOFLOW_API_KEY = "본인의 API-KEY 입력" # 본인의 API 키를 입력하세요.
WORKSPACE_ID = "jiyeonjin"
PROJECT_ID = "segmentation_-b4buk"
VERSION_NUMBER = 1

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(WORKSPACE_ID).project(PROJECT_ID)
dataset = project.version(VERSION_NUMBER).download("png-mask-semantic")
base_data_path = dataset.location
clear_output()
print(f"✅ 데이터셋 다운로드 완료! 경로: {base_data_path}")


# ----------------- [ 2. 데이터셋 준비 ] -----------------
final_datasets = DatasetDict()
for split in ["train", "valid", "test"]:
    split_path = os.path.join(base_data_path, split)
    if not os.path.isdir(split_path): continue
    image_paths = sorted(glob.glob(os.path.join(split_path, "*.jpg")))
    label_paths = [img_path.replace(".jpg", "_mask.png") for img_path in image_paths]
    ds = Dataset.from_dict({"image": image_paths, "label": label_paths})
    final_datasets[split] = ds.cast_column("image", HFImage()).cast_column("label", HFImage())

train_dataset = final_datasets["train"]
valid_dataset = final_datasets["valid"]
print("✅ 데이터셋 준비 완료!")


# ----------------- [ 3. 모델, 프로세서 및 데이터 변환 설정 (최종 수정) ] -----------------
id2label = {0: "background", 1: "lane"}
model_checkpoint = "nvidia/segformer-b0-finetuned-ade-512-512"
processor = SegformerImageProcessor.from_pretrained(model_checkpoint, do_reduce_labels=False)
model = SegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint, num_labels=len(id2label), id2label=id2label, label2id={v: k for k, v in id2label.items()}, ignore_mismatched_sizes=True
)

jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

# --- [ Processor를 믿지 않는, 가장 강력한 최종 변환 함수 ] ---
def final_transforms(example_batch):
    # 1. 이미지와 라벨을 평소처럼 준비
    images = [jitter(x.convert("RGB")) for x in example_batch["image"]]
    labels = [mask.convert("L") for mask in example_batch["label"]] # 흑백으로 변환

    # 2. 일단 프로세서에게 변환을 맡김
    inputs = processor(images, labels, return_tensors="pt")

    # 3. [핵심] 프로세서가 만든 최종 'labels' 텐서를 직접 검사하고 클렌징
    #    0보다 작거나 1보다 큰 값이 있다면 모두 1로 강제 변경
    inputs['labels'][inputs['labels'] > 1] = 1
    inputs['labels'][inputs['labels'] < 0] = 0

    return inputs

train_dataset.set_transform(final_transforms)
valid_dataset.set_transform(final_transforms)

print("✅ 모델, 프로세서 및 최종 텐서 클렌징 설정 완료.")


# ----------------- [ 4. 모델 훈련 ] -----------------
training_args = TrainingArguments(
    output_dir="segformer-b0-finetuned-lanes-final-v7",
    learning_rate=0.00006,
    num_train_epochs=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_total_limit=1,
    save_steps=200,
    eval_steps=200,
    logging_steps=10,
    remove_unused_columns=False,
)

trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=valid_dataset)

print("\n✅ 모델 훈련을 시작합니다...")
trainer.train()
print("✅ 모델 훈련 완료!")


# ===================================================================
# [ 5. 영상 업로드 및 처리 (수정된 최종본) ]
# ===================================================================

uploader = widgets.FileUpload(accept='.mp4', multiple=False, description='영상 업로드')
print("\n✅ 훈련된 모델을 사용할 준비가 되었습니다. 처리할 영상을 업로드하세요.")
display(uploader)

def process_video_final(model, processor, uploaded_file, output_filename="output_video.mp4"):
    if not uploaded_file: return
    # 여러 파일이 업로드 될 수 있는 uploader의 값(튜플)에서 첫번째 파일(딕셔너리)을 선택합니다.
    file_info = uploaded_file[0]
    input_bytes = file_info['content']
    input_filename = "input_video.mp4"
    with open(input_filename, 'wb') as f: f.write(input_bytes)
    video_capture = cv2.VideoCapture(input_filename)
    fps, total_frames = video_capture.get(cv2.CAP_PROP_FPS), int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    w, h = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_writer = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    model.to(device)
    print(f"\n✅ 영상 처리를 시작합니다... (총 {total_frames} 프레임)")
    for _ in tqdm(range(total_frames)):
        ret, frame = video_capture.read()
        if not ret: break
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        with torch.no_grad():
            inputs = processor(images=image, return_tensors="pt").to(device)
            logits = model(**inputs).logits.cpu()
        mask = torch.nn.functional.interpolate(logits, size=(h, w), mode="bilinear", align_corners=False).argmax(dim=1)[0].numpy().astype(np.uint8)
        color_mask = np.zeros_like(frame)
        color_mask[mask == 1] = [0, 255, 0]
        overlaid_frame = cv2.addWeighted(frame, 1, color_mask, 0.5, 0)
        video_writer.write(overlaid_frame)
    video_capture.release()
    video_writer.release()
    os.remove(input_filename)
    print(f"\n✅ 영상 처리 완료! 결과가 '{output_filename}' 파일로 저장되었습니다.")

# [중요] 자동 실행 기능이 제거되었습니다.
# 이제 이 셀은 영상 처리 함수를 정의하고, 업로드 버튼을 보여주는 역할만 합니다.

✅ 데이터셋 다운로드 완료! 경로: /workspace/segmentation_-1
✅ 데이터셋 준비 완료!


The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type', 'reduce_labels'
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([2]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([2, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 모델, 프로세서 및 최종 텐서 클렌징 설정 완료.

✅ 모델 훈련을 시작합니다...


Step,Training Loss
10,0.679500
20,0.608100
30,0.540400
40,0.480200
50,0.420900
60,0.373400
70,0.326500
80,0.283800
90,0.251400
100,0.221800


✅ 모델 훈련 완료!

✅ 훈련된 모델을 사용할 준비가 되었습니다. 처리할 영상을 업로드하세요.


FileUpload(value=(), accept='.mp4', description='영상 업로드')

In [ ]:
#***********************************************************************************************************************************************************************


# 5번 단계까지 실행했는데 영상 파일이 업로드 되지 않는 오류 발생. 따라서 아래 솔루션 이용

# 불안정한 업로드 위젯을 완전히 우회하고, Jupyter 환경의 파일 탐색기를 통해 직접 파일을 업로드하는 방법으로 변경.
# 4번 단계([ 4. 모델 훈련 ])는 다시 실행하게 되면 에포크가 재진행되니(15분 정도 소요) 한 번 실행시켰으면 이후로 실행시키지 말것. 이후의 셀들은 4번 단계를 다시 실행시키지 않아도 정상적으로 실행됨.
# 오류를 해결하기 위해 바로 아래의 [ ★ 새로운 셀 ★ ] 실행시키기.
# Jupyter 노트북 환경에서 Upload Files를 이용해 훈련시키고 싶은 원본 영상 업로드 하기.
# 원본 영상이 성공적으로 업로드 됐다면 [ ★ 최종 해결용 셀 ★ ] 실행시키고 결과 영상 다운받기.


#***********************************************************************************************************************************************************************
#이 내용들은 runpods Jupyter 환경에서 작업하였습니다.

In [ ]:
# ===================================================================
# [ ★ 새로운 셀 ★ ] - 저장된 모델 안전하게 불러오기
# ===================================================================
import glob

# 4번 셀을 다시 실행할 필요 없이, 훈련이 끝난 모델을 폴더에서 직접 불러옵니다.
# 가장 마지막에 저장된 체크포인트 폴더를 자동으로 찾아줍니다.
try:
    last_checkpoint = sorted(glob.glob("./segformer-b0-finetuned-lanes-final-v7/checkpoint-*/"))[-1]
    model = SegformerForSemanticSegmentation.from_pretrained(last_checkpoint)
    processor = SegformerImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512", do_reduce_labels=False)
    print(f"✅ 훈련이 완료된 모델 ({last_checkpoint})을 성공적으로 불러왔습니다.")
except IndexError:
    print("❌ 오류: 저장된 체크포인트 폴더를 찾을 수 없습니다. 훈련이 정상적으로 완료되었는지 확인해주세요.")

✅ 훈련이 완료된 모델 (./segformer-b0-finetuned-lanes-final-v7/checkpoint-4000/)을 성공적으로 불러왔습니다.


In [ ]:
# ===================================================================
# [ ★ 최종 해결용 셀 ★ ] - 직접 업로드된 파일 처리하기
# ===================================================================

# 1. 여기에 방금 직접 업로드한 영상 파일의 정확한 이름을 입력하세요.
#    (예: "내영상.mp4" 또는 "test_video.mp4")
video_filename = "원본영상.mp4"


# 2. 아래 코드가 위 파일 이름으로 영상 처리를 시작합니다.
print(f"✅ '{video_filename}' 파일을 직접 처리합니다. 잠시만 기다려주세요...")

# --- 기존의 process_video_final 함수와 거의 동일한 로직 ---
try:
    # 모델이 메모리에 있는지 확인
    model.eval()

    input_filename = video_filename
    output_filename="final_output_video.mp4"

    video_capture = cv2.VideoCapture(input_filename)
    if not video_capture.isOpened():
        print(f"❌ 오류: '{input_filename}' 파일을 열 수 없습니다. 파일 이름이 정확한지, 파일이 손상되지 않았는지 확인하세요.")
    else:
        fps = video_capture.get(cv2.CAP_PROP_FPS)
        total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
        w = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

        video_writer = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        print(f"\n✅ 영상 처리를 시작합니다... (총 {total_frames} 프레임)")

        for _ in tqdm(range(total_frames)):
            ret, frame = video_capture.read()
            if not ret: break
            image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            with torch.no_grad():
                inputs = processor(images=image, return_tensors="pt").to(device)
                logits = model(**inputs).logits.cpu()

            mask = torch.nn.functional.interpolate(logits, size=(h, w), mode="bilinear", align_corners=False).argmax(dim=1)[0].numpy().astype(np.uint8)

            color_mask = np.zeros_like(frame)
            color_mask[mask == 1] = [0, 255, 0]

            overlaid_frame = cv2.addWeighted(frame, 1, color_mask, 0.5, 0)
            video_writer.write(overlaid_frame)

        video_capture.release()
        video_writer.release()

        print(f"\n✅ 영상 처리 완료! 결과가 '{output_filename}' 파일로 저장되었습니다.")

except NameError:
    print("❌ 오류: 'model' 변수를 찾을 수 없습니다. 커널 재시작 후 [0, 1, 2, 3] 셀과 [★저장된 모델 불러오기★] 셀을 실행했는지 확인해주세요.")
except Exception as e:
    print(f"❌ 예상치 못한 오류가 발생했습니다: {e}")

✅ '원본영상.mp4' 파일을 직접 처리합니다. 잠시만 기다려주세요...

✅ 영상 처리를 시작합니다... (총 2522 프레임)


  0%|          | 0/2522 [00:00<?, ?it/s]


✅ 영상 처리 완료! 결과가 'final_output_video.mp4' 파일로 저장되었습니다.
